# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy import io
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cylouvain
import networkx as nx
from scipy.interpolate import make_interp_spline
import matplotlib as mpl
import random
from sklearn.decomposition import PCA
import seaborn as sns
import math
from sklearn.cross_decomposition import CCA
from scipy import linalg
import copy
import umap

#!git clone https://github.com/NeuralAnalysis/PyalData
from PyalData.pyaldata import *


# for LSTM
import xarray as xr
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.decoders import LSTMClassification
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

# for Bayes Classifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Load and Preprocessing

## Load dataset

In [6]:
def load_dataset_from_mat(file_path: str):
    '''
    @arg file_path - path to file in .mat format
    @returns data_pd - dataset as pandas DataFrame  
    '''

    data_pd = mat2dataframe(file_path, shift_idx_fields=True)
    return data_pd

## Preprocessing - waveform data

In [9]:
def filter_trial_ID(data_waveform):
    '''
    remove units (neurons) with ID 0 or ID 255 from waveform dataset (depends on software used to record)
    
    @arg data_waveform - waveform dataset, type pandas DataFrame
    @return filtered_data - filtered waveform dataset
    '''
    trials_to_keep = []
    for i in range(data_waveform.shape[0]):
        if(data_waveform.ID[i] != 0 and data_waveform.ID[i] != 255): 
            if np.ndim(data_waveform.spikes.iloc[i]) == 1: print(i) # to identify bugs
            if data_waveform.chan[i] > 96: print(i) # to identify bugs
            trials_to_keep.append(i)
   
    waveform_filtered = data_waveform.loc[trials_to_keep, :] 
    return waveform_filtered


def filter_SNR(waveform_filtered, data_spikes, SNR):
    '''
    filter waveform dataset, keep only waveforms with minimal SNR
    
    @arg waveform_filtered - waveform dataset, type pandas DataFrame
    @return waveform_filtered - filtered waveform dataset
    @return trials_to_keep_M1 - list containing index of trials belonging to M1 neurons
    @return trials_to_keep_PMd - list containing index of trials belonging to PMd neurons
    '''
    
    trials_to_keep = []
    trials_to_keep_M1 = []
    trials_to_keep_PMd = []

    for j in range(waveform_filtered.shape[0]): # for each neuron (each row)
        b = []
        for i in range(waveform_filtered.spikes.iloc[j].shape[0]): # for each spike from neuron j (each row in spikes = waveform)
            AP = waveform_filtered.spikes.iloc[j][i,:] # save here the AP to make it faster
            peak = np.amax(waveform_filtered.spikes.iloc[j][i,1:])
            trough = np.min(AP)
            std = np.std(AP)
            a = (peak - trough)/std # snr of each spike from neuron j
            b.append(a) # [snr_spike0, snr_spike1, snr_spike2, ..., snr_spikeEND]
        b = np.array(b)
        snr = np.mean(b) # average SNR for neuron j

        if(snr > SNR):# if snr of neuron j is > SNR:
            trials_to_keep.append(j) # save index of neurons to keep

            if j <= data_spikes.M1_spikes[0].shape[1]-1: # j=neuron_ID <= number of neurons in M1 in trial
                trials_to_keep_M1.append(j) # to compare with M1_unit_guide
            else:
                trials_to_keep_PMd.append(j-data_spikes.M1_spikes[0].shape[1])#it was shape[1]-1 # to compare with PMd_unit_guide

    waveform_filtered = waveform_filtered.iloc[trials_to_keep, :] # same structure than original waveform dataset but only neurons with snr>3
        
    return waveform_filtered, trials_to_keep_M1, trials_to_keep_PMd


def concatenate_average_trials(waveform_filtered):
    '''
    concatenate average of each trial
    
    @arg data - waveform dataset, type pandas DataFrame, already filtered
    @return concatenated_avg - concatenated array
    '''
    
    concatenate_avg = []
    for j in range(waveform_filtered.shape[0]): # for each neuron
        avg = np.mean(waveform_filtered.spikes.iloc[j], axis=0) # average among all waveforms across time (= across all rows) -> each channel has only the average spike
        b = np.array(avg)
        b = np.delete(b,0)  
        concatenate_avg.append(b) # concatenate all averaged spikes [[average spike of channels 0], [average spike of channels 1], ...]
    concatenate_avg = np.array(concatenate_avg)
    
    return concatenate_avg


def normalize_data(concatenated_data, plot=False):
    '''
    Normalize each waveform between -1 and 1
    
    @arg concatenated_data - concatenated waveforms from all trials
    @return data_normalized - list of all normalized waveforms
    '''
    data_normalized = []

    for i in range(concatenated_data.shape[0]):
        wave = concatenated_data[i]

        d = 2.*(wave - np.min(wave))/np.ptp(wave)-1 # normalization between -1 and 1 of each waveform
        data_normalized.append(d)
        
        if plot:
            plt.plot(data_normalized[i], c = 'blue')

    if plot:
        plt.title('Normalized waveform with respect to the neuron activity')
        plt.ylim((-1.2,1.2))
        plt.show()
    
    return data_normalized




## Preprocessing - spikes data

In [ ]:
def preprocessing_general(data_spikes, combine=True, std=0.05):
    '''
    process spike data, the following steps are done:
        - combine bins
        - sqrt transform
        - filter only successful triasl
        - convert firing rates column
        
    @arg data_spikes - firing data, type: pandas DataFrame 
    @arg combine - Bool, if True, combine bins
    @arg std - std defined for smoothing function
    
    @return processed firing data, type: pandas DataFrame 
    '''
    if combine:
        td = combine_time_bins(data_spikes, 2) #merge 2 bins -> 20 ms bins
    else:
        td = data_spikes
    td = transform_signal(td, "M1_spikes",  'sqrt')
    td = transform_signal(td, "PMd_spikes", 'sqrt')
    td = merge_signals(td, ["M1_spikes", "PMd_spikes"], "both_spikes")
    td = add_firing_rates(td, 'smooth', std=std) # add M1_rates and PMd_rates fields -> rate = spikes/bin_size
    td = select_trials(td, "result == 'R'")
    return td

def preprocessing_spikes(td, trials_to_keep_M1, trials_to_keep_PMd, start_point_name, rel_start, rel_end, rates=True):
    '''
    preprocess 2
    
    @arg td - preprocessed firing data, type: pandas DataFrame 
    @arg trials_to_keep_M1 - list of neurons to keep from M1
    @arg trials_to_keep_PMd - list of neurons to keep from PMd
    @arg rates - if True, subtract cross condition mean (because this filtered dataset will be used to work with rates)
    @arg rel_start, rel_end - when to start/end extracting relative to the starting time point (restrict interval)
    @start_point_name - name of the time point around which the interval starts
    
    @return processed and filtered firing data, type: pandas DataFrame 
    '''
    
    td_processed = restrict_to_interval(td, start_point_name, rel_start=rel_start, rel_end=rel_end)
    
    # this part of preprocessing depends on if we look at spikes or at rates
    if rates:
        td_processed = subtract_cross_condition_mean(td_processed) #Find mean across all trials for each time point and subtract it from each trial.
    
    # only save data of neurons that had SNR >3 (found before)
    for j in range(td_processed.shape[0]):
        td_processed.M1_spikes[j] = td_processed.M1_spikes[j][:, trials_to_keep_M1] # all rows corresponding to neurons with SNR>3
        td_processed.PMd_spikes[j] = td_processed.PMd_spikes[j][:, trials_to_keep_PMd] # all rows corresponding to neurons with SNR>3

    # unit_guide[i] has shape #neurons x 2, ex M1_unit_guide -> 76x2 (dataset 1)
    # why this only for first trial and not for all of the trials?
    td_processed.M1_unit_guide[0] = td_processed.M1_unit_guide[0][trials_to_keep_M1, :] # only get the neurons (both columns) with SNR >3
    td_processed.PMd_unit_guide[0] = td_processed.PMd_unit_guide[0][trials_to_keep_PMd, :] # only get the neurons (both columns) with SNR >3
    
    td_processed = remove_low_firing_neurons(td_processed, "M1_spikes", 1)
    td_processed = remove_low_firing_neurons(td_processed, "PMd_spikes", 1)
    td_processed["target_id"] = td_processed.apply(get_target_id, axis=1)
    
    return td_processed

# UMAP + Clustering

In [1]:
def umap_on_waveform(data_normalized, neighbors, seed, n_components=2, plot=True):
    '''
    perform UMAP
    
    @arg data_normalized - normalised waveform dataset, type pandas DataFrame
    @arg neighbors_ - number of neighbours to perform UMAP
    @arg seed_ - random seed for UMAP
    @arg n_components - number of dimension for resulting UMAP graph
    @arg plot - if True plot UMAP graph 
    
    @return umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                      graph and corresponding x and y position
    @return mapper - UMAP output (needed for Louvian clustering)
    @return embedding - array, eavh row is a neuron represented in the UMAP graph
    
    '''
    # UMAP
    reducer = umap.UMAP(n_neighbors = neighbors, min_dist=0.1, random_state=seed, n_components=n_components) # old -> n_neighbors=30
    mapper = reducer.fit(data_normalized)
    embedding = reducer.transform(data_normalized)

    if plot:
        # Each row of the embedding array is a 2D/3D representation of the corresponding unit(=neuron)
        if n_components == 3:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], s=20,  c = '#75759E')
        else:
            plt.scatter(embedding[:,0], embedding[:,1], s=20,  c = '#75759E')

    # save all normalised data into a DataFrame with corresopnding position in umap
    if n_components == 3:
        umap_df = pd.DataFrame(embedding, columns=('x', 'y', 'z')) # create DataFrame with 2 columns: x,y coordinates of UMAP diagram
    else:
        umap_df = pd.DataFrame(embedding, columns=('x', 'y')) # create DataFrame with 2 columns: x,y coordinates of UMAP diagram        
    umap_df['waveform'] = list(data_normalized) # add to dataframe a column with waveform corresponding to that point
    
    #umap.plot.connectivity(mapper, show_points=True)
    return umap_df, mapper, embedding


def louvian_clustering(mapper, umap_df, CUSTOM_PAL_SORT, res, n_components=2, plot=True):
    '''
    perform Louvain Community Detection
    
    @arg mapper - UMAP output, type umap.umap_.UMAP
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                      graph and corresponding x and y position
    @arg CUSTOM_PAL_SORT - list of colors for each cluser
    @arg res - resolution for Louvian clustering
    @arg n_components - number of dimension from UMAP graph
    @arg plot - if True plot UMAP graph 
    
    @return umap_df - pandas DataFrame same as input but with added cluster each point belongs to
    @return clustering_solution - list of clusters each neuron belongs to
    '''
    # Louvian clustering
    G = nx.from_scipy_sparse_matrix(mapper.graph_) # build graph, every node is a points on UMAP graph
    clustering = cylouvain.best_partition(G, resolution = res) # returns dict {'node': cluster it belongs to}
    clustering_solution = list(clustering.values()) # number of community (cluster) of each node
    umap_df['cluster'] = clustering_solution
    
    if plot:
        cluster_colors = [CUSTOM_PAL_SORT[i] for i in clustering_solution]
        if n_components == 3:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(umap_df['x'].tolist(), umap_df['y'].tolist(),umap_df['z'].tolist(), marker='o', c=cluster_colors, s=20,linewidth=0.5)
        else:
            plt.figure(figsize=(10, 7))
            plt.scatter(umap_df['x'].tolist(), umap_df['y'].tolist(), marker='o', c=cluster_colors, s=50,linewidth=0.5)
    
    return umap_df, clustering_solution



# Functions - general

In [ ]:
# only used to check difference with previous results
def neurons_in_cluster(clustering_solution):
    '''
    compute number of neurons in each cluster
    
    @arg clustering_solution - list of clusters each neuron belongs to
    
    @return neurons - list of number of neurons in each class
    '''
    neurons = [0]*(max(clustering_solution)+1)

    for j in range(max(clustering_solution)+1): # for each cluster
        for i in range(len(clustering_solution)): #for each neuron=node in UMAP graph
            if(clustering_solution[i] == j): # count neurons in cluster 7
                  neurons[j] += 1
    return neurons

In [ ]:
def neuron_classes(clustering_solution):
    '''
    assign number (ID) of neuron to the class it belongs to
    
    @arg clustering_solution - list of clusters each neuron belongs to
    
    @return neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    '''
    neuron_types = {}
    for i in range(len(set(clustering_solution))):
        type_i = []
        for j in range(len(clustering_solution)):
            if(clustering_solution[j] == i):
                type_i.append(j)
        neuron_types[i] = type_i
    return neuron_types



def sample_neurons(neuron_types, N_samples):
    '''
    sample ranodm neurons from each cluster
    
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    @arg N_samples - number of neurons we want to sample from each cluster
    
    @return sampled_data - dict: key is cluster, values are list with sampled neurons belonging to that cluster
    '''
    # neuron_types: dictionary -> {cluster_number: [id=number of all neurons in this cluster], ...}
    # N_samples: number of neurons to sample from each neuron-class
    neuron_types_copy = copy.deepcopy(neuron_types)
    sampled_data = {}
    for i in range(len(neuron_types)): # for each cluster
        type_index = []
        for j in range(N_samples): # select x neurons from each cluster
            random_neuron = random.choice(neuron_types_copy[i]) # pick a random neuron from cluster i
            type_index.append(random_neuron)
            neuron_types_copy[i].remove(random_neuron) # remove already picked neuron so they don't appear twice
        sampled_data[i] = type_index[0:N_samples] # 22 neurons for each cluster

    return sampled_data

# Cluster Properties

## PCA

In [ ]:
def PCA_variance_explained(td_processed, neuron_types, vline, CUSTOM_PAL_SORT, hline=True):
    '''
    plot variance explained ratio as a function of PCA number of dimensions for each cluster
    
    @arg td_processed - preprocessed and filtered firing data
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    @arg vline - where to plot vertical line on graph
    @arg CUSTOM_PAL_SORT - personalised colour palette
    @arg hline - if True plot horizontal line corresponding to 85% variance explained
    '''

    for j in range(len(neuron_types)):
        data_spikes_new = merge_signals(td_processed, ["M1_rates", "PMd_rates"], "both_rates")

        for i in range(data_spikes_new.shape[0]):
            data_spikes_new.both_rates[i] = data_spikes_new.both_rates[i][:, neuron_types[j]]

        # PCA
        pca_dims = len(neuron_types[j])
        pca = fit_dim_reduce_model(data_spikes_new, PCA(pca_dims), "both_rates")

        for i in range(1, len(pca.explained_variance_ratio_ )):
            pca.explained_variance_ratio_[i] = pca.explained_variance_ratio_[i] + pca.explained_variance_ratio_[i-1]

        #t = list(range(1,pca.explained_variance_ratio_.shape[0]+1))
        #plt.plot(t,pca.explained_variance_ratio_, c= CUSTOM_PAL_SORT[j], label=f'{j}')
        t = list(range(0,pca.explained_variance_ratio_.shape[0]+1))
        plt.plot(t,np.insert(pca.explained_variance_ratio_,0,0), c= CUSTOM_PAL_SORT[j], label=f'{j}')
        plt.scatter(t,np.insert(pca.explained_variance_ratio_,0,0),marker='.', facecolors='none', edgecolors= CUSTOM_PAL_SORT[j], s=25)
    
    max_line = max([len(neuron_types[i]) for i in range(len(neuron_types))])
    plt.legend()
    plt.xlabel('Number of dimensions')
    plt.ylabel('Variance explained')
    plt.vlines(vline, 0, 1, color='k', linewidth=1, linestyle='--')
    if hline:
        plt.hlines(0.85, 0, max_line, color='k', linewidth=1, linestyle='--')
        
    plt.show()
    
    return

def apply_PCA(td_processed, sampled_data, cluster, pca_dims, return_model=False):
    '''
    apply PCA to a sub-sampled set of data of a specific neuron cluster
    
    @arg td_processed - processed and filtered firing data, type: pandas DataFrame 
    @arg sampled_data - sampled neurons from each cluster
    @arg cluster - (int) cluster to apply PCA on.
                    If cluster=-1: sampled_data contains only sampled neurons 
    @arg pca_dims - number of dimensions for PCA
    @return_model - if True return also model
    
    @return move_td - dataset (pandas DataFrame) with added field with pca output
    @return model - dimensionality reduction model
    '''
    
    if cluster != -1:
        sampled_data = sampled_data[cluster]
        
    move_td3 = merge_signals(td_processed, ["M1_rates", "PMd_rates"], "both_rates")
    for i in range(move_td3.shape[0]):
        move_td3.both_rates[i] = move_td3.both_rates[i][:, sampled_data]

    # apply PCA
    if return_model:
        move_td, model = dim_reduce(move_td3, PCA(pca_dims), "both_rates", "both_pca", return_model=return_model)
        return move_td, model
    else:
        move_td = dim_reduce(move_td3, PCA(pca_dims), "both_rates", "both_pca", return_model=return_model)
        return move_td

In [ ]:
# not used
def orthogonality_prep_mov(td_processed, sampled_data, cluster, pca_dims=10):
    
    data_spikes_merged = merge_signals(td_processed, ["M1_rates", "PMd_rates"], "both_rates")
    for i in range(data_spikes_merged.shape[0]):
          data_spikes_merged.both_rates[i] = data_spikes_merged.both_rates[i][:, sampled_data[cluster]]
 
    pca = fit_dim_reduce_model(data_spikes_merged, PCA(pca_dims), "both_rates")


## dPCA - not used

In [ ]:
# not used
def apply_dPCA(td_processed, tr_per_target, N_samples, cluster):
    '''
    apply dPCA to a sub-sampled set of data from all trials and targets
    
    @arg td_processed - processed and filtered firing data, type: pandas DataFrame 
    @arg tr_per_target - 2D array containing trials id of each trial going to each specific target
    @arg N_samples - number of neurons to sample from each cluster
    @arg cluster - (int) cluster to apply dPCA on.
                    If cluster=-1: sampled neurons from all dataset, not from one cluster only
    @return 
    
    '''
    num_trials = td_processed.shape[0]
    num_neurons = td_processed.both_spikes[0].shape[1]
    num_targets = len(tr_per_target)
    min_per_target = np.min(tr_per_target)
    num_time = td_processed.both_spikes[0].shape[0]

     
    if cluster == -1:
        sampled_data = np.random.choice(np.arange(num_neurons), size=N_samples) # sample from whole dataset
    else:
        sampled_data = sample_neurons(neuron_types, N_samples)
        
    data = np.zeros((min_per_target, N_samples, num_targets, num_time))
    tr_per_target_found = [0]*num_targets

    for trial in range(num_trials):
        targetID = td_processed.target_id[trial] # ID of target of this trial
        if tr_per_target_found[targetID] >= min_per_target: continue
        data[tr_per_target_found[targetID], :, targetID, :] = np.transpose(td_processed.both_spikes[trial][:,sampled_data[cluster]])
        tr_per_target[targetID] += 1 # trials for this target already found

    avg_trial_per_target = np.mean(data, axis=0)
    
    dpca = dPCA.dPCA(labels='st',regularizer='auto')
    dpca.protect = ['t']
    Z = dpca.fit_transform(avg_trial_per_target, data)
    
    return Z
    

In [1]:
# not used
def dPCA_transformation_matrix(td_processed, tr_per_target, sampled_data, N_samples, cluster):
    '''
    .........
    '''
    # sample neurons

    # we want to reshape data according to the following:
    # [num of trials in each target, num of neurons, num of targets, time point]
    num_trials = td_processed.shape[0]
    num_neurons = td_processed.both_spikes[0].shape[1]
    num_targets = len(tr_per_target)
    min_per_target = np.min(tr_per_target)
    num_time = td_processed.both_spikes[0].shape[0]

    #neuron_types = neuron_classes(clustering_solution) # neuron in each cluster

    Z_all = {}
    transf_matrix_t = {}
    transf_matrix_s = {}
    transf_matrix_st = {}
    for clust in range(len(neuron_types)):
            Z_all[f'{clust}'] = []
            transf_matrix_t[f'{clust}'] = []
            transf_matrix_s[f'{clust}'] = []
            transf_matrix_st[f'{clust}'] = []

    #for cluster in range(len(neuron_types)):

    #sampled_data = sample_neurons(neuron_types, N_samples)
    data = np.zeros((min_per_target, N_samples, num_targets, num_time))
    tr_per_target_found = [0]*num_targets

    # reshape data to use it in the dPCA function
    for trial in range(num_trials):
        targetID = td_processed.target_id[trial] # ID of target of this trial
        if tr_per_target_found[targetID] >= min_per_target: continue
        data[tr_per_target_found[targetID], :, targetID, :] = np.transpose(td_processed.both_spikes[trial][:,sampled_data[cluster]])
        tr_per_target_found[targetID] += 1 # trials for this target already found
    avg_trial_per_target = np.mean(data, axis=0)

    # apply dPCA 
    dpca = dPCA.dPCA(labels='st',regularizer='auto')
    dpca.protect = ['t']
    Z_all = dpca.fit_transform(avg_trial_per_target, data)

    # compute inverse of avg data
    data_avg_inv = np.linalg.pinv(avg_trial_per_target)

    '''
    # compute transformation matrices for each cluster
    transf_matrix_t[f'{cluster}'] = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all[f'{cluster}']['t'],[1,2,0]))
    transf_matrix_s[f'{cluster}'] = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all[f'{cluster}']['s'],[1,2,0]))
    transf_matrix_st[f'{cluster}'] = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all[f'{cluster}']['st'],[1,2,0]))
    '''
    transf_matrix_t = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all['t'],[1,2,0]))
    transf_matrix_s = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all['s'],[1,2,0]))
    transf_matrix_st = np.matmul(np.transpose(data_avg_inv,[2,0,1]),np.transpose(Z_all['st'],[1,2,0]))
    
    return transf_matrix_t, transf_matrix_s, transf_matrix_st
    

In [ ]:
# not used
def dpca_explained_variance(td_processed, tr_per_target, neuron_types, n_components):
    # all neurons from each cluster

    # we want to reshape data according to the following:
    # [num of trials in each target, num of neurons, num of targets, time point]
    num_trials = td_processed.shape[0]
    num_neurons = td_processed.both_spikes[0].shape[1]
    num_targets = len(tr_per_target)
    min_per_target = np.min(tr_per_target)
    num_time = td_processed.both_spikes[0].shape[0]

    Z_all = {}
    explained_variance_ratio = {}
    
    for clust in range(len(neuron_types)):
            Z_all[f'{clust}'] = []
            explained_variance_ratio[f'{clust}'] = []

    for cluster in range(len(neuron_types)):

        data = np.zeros((min_per_target, len(neuron_types[cluster]), num_targets, num_time))
        tr_per_target_found = [0]*num_targets

        for trial in range(num_trials):
            targetID = td_processed.target_id[trial] # ID of target of this trial
            if tr_per_target_found[targetID] >= min_per_target: continue
            data[tr_per_target_found[targetID], :, targetID, :] = np.transpose(td_processed.both_spikes[trial][:,neuron_types[cluster]])
            tr_per_target_found[targetID] += 1 # trials for this target already found

        avg_trial_per_target = np.mean(data, axis=0)

        # apply dPCA 
        dpca = dPCA.dPCA(labels='st',regularizer='auto', n_components=n_components)
        dpca.protect = ['t']
        
        Z_all[f'{cluster}'] = dpca.fit_transform(avg_trial_per_target, data)
        explained_variance_ratio[f'{cluster}'] = dpca.explained_variance_ratio_
    
    return explained_variance_ratio


## Latent variables

In [1]:
def get_target_id(trial):
    '''
    convert direction from dataset to target ID 
    
    @arg trial - number of trial from dataset (int)
    
    @return target ID (int)
    '''
    return int(np.round((trial.target_direction + np.pi) / (0.25 * np.pi))) - 1


def trials_per_target(td_processed):
    '''
    select trials going to each target
    
    @arg td_processed - preprocessed and filtered firing data
    
    @return 2D list, each entry is a list of all trials going to specific target
    '''
    targets = td_processed.target_id.unique() # [0, 1, 2, 3, 4, 5, 6, 7]
    targets.sort(axis=0)  # ascending order

    # separate by target
    # each list contains indices of trials going to target __
    target_0_index = []
    target_1_index = []
    target_2_index = []
    target_3_index = []
    target_4_index = []
    target_5_index = []
    target_6_index = []
    target_7_index = []

    for i in range(td_processed.shape[0]):
        if(td_processed.target_id[i] == 0):
            target_0_index.append(i)
        if(td_processed.target_id[i] == 1):
            target_1_index.append(i)
        if(td_processed.target_id[i] == 2):
            target_2_index.append(i)
        if(td_processed.target_id[i] == 3):
            target_3_index.append(i)
        if(td_processed.target_id[i] == 4):
            target_4_index.append(i)
        if(td_processed.target_id[i] == 5):
            target_5_index.append(i)
        if(td_processed.target_id[i] == 6):
            target_6_index.append(i)
        if(td_processed.target_id[i] == 7):
            target_7_index.append(i)

    # [[indices of trials going to taregt 1], [indices of trials going to taregt 2],...]
    target_index = [target_0_index, target_1_index, target_2_index, target_3_index, target_4_index, target_5_index, target_6_index, target_7_index]
    
    return target_index


def get_lat_variables_per_target(target, data, dim, target_index):
    '''
    get latent variables for each target -> average response of all neurons when going to specific target
    
    @arg target - number of target (int from 0 to 7)
    @arg data - firing rates output from PCA
    @arg dim - slected number of dimensions for PCA
    
    @return latent variable, 2D list of size _time_interval_ x _PCA_dimensions_
    
    '''
    time_interval = data[0].shape[0] # time-steps left after applying combine_time_bins() and restrict_to_interval()
    sum_per_target = np.zeros((time_interval, dim)) 

    for trial in target_index[target]: # for each trial going to given target 
        sum_per_target  = sum_per_target  + data[trial] # sum 

    latent_variable = sum_per_target /len(target_index[target]) # compute average response for this target
    
    return latent_variable 


def get_all_latent_variables(td_processed, neuron_types, pca_dims, target_index):
    '''
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    @arg td_processed - preprocessed and filtered firing rates data
    @arg pca_dims - number of dimensions chosen for PCA (should change for each cluster?)
    @ arg target_index - 2D list, each entry is a list of all trials going to specific target
    
    @return dict: key is cluster, values are latent variables for all targets for that class
    '''
    
    clusters = len(neuron_types)
    num_targets = np.shape(target_index)[0]
    
    lat_variables = {} # key is cluster number, values are latent variables for that cluster for all targets
    for clust in range(clusters):
        lat_variables[f'{clust}'] = []

    for clust in range(clusters):
        spikes_data = merge_signals(td_processed, ["M1_rates", "PMd_rates"], "both_rates")
        for i in range(spikes_data.shape[0]):   
            spikes_data.both_rates[i] = spikes_data.both_rates[i][:, neuron_types[clust]]

        # PCA
        spikes_data = dim_reduce(spikes_data, PCA(pca_dims), "both_rates", "both_pca")
        
        for target in range(num_targets):  
            lat_variables[f'{clust}'].append(get_lat_variables_per_target(target, spikes_data.both_pca, pca_dims, target_index))

    return lat_variables


def plot_latent_variables(lat_variables, target_index, cluster, l_var, ax):
    '''
    plot one latent variable for a specific cluster
   
    
    @arg lat_variables - dictwith all latent variables:
                        key is cluster, values are latent variables for all targets for that class.
    @arg cluster - cluster of latent variable to plot
    @arg l_var - specific latent variable to be plotted
    @arg ax - axis where to plot
    
    to access specific latent variable 
    lat_variables['0'][1][:,2] # cluster 0, going to target 1, latent variable 2 (: indicates all time steps)
    '''
    lat_var = []
    num_targets = np.shape(target_index)[0]

    for target in range(num_targets):
        lat_var.append(lat_variables[f'{cluster}'][target][:,l_var]) # latent variable for one cluster including all targets

    # get all altent variables for one cluster
    for i in range(len(lat_var)): # for each target
        ax.plot(lat_var[i][:],label=f'target {i}') 
    #ax.legend(bbox_to_anchor=(0.5,-0.05))
    ax.set_title(f'cluster {cluster}, latent variable {l_var}')
    
    return

## Firing rate

In [ ]:
def firing_rate_per_cluster(td_processed, clustering_solution):
    '''
    calculate firing rate of each neuron and average firing rate of each cluster
    
    @arg td_processed - preprcessed SPIKES data (not rates)
    @clustering_solution - list of clusters each neuron belongs to
    
    Note SME = standard error of a sample mean
    
    @return overall_FR - average firing rate across all neurons and all trials
    @return overall_FR_sme - SME  across all neurons and all trials
    @return avg_FR - average firing rate of each cluster
    @return sme_FR - SME of firing rate of each cluster
    @return max_FR - max firing rate of each cluster
    '''
    
    #fr_neurons -> avergae firing rate of each neuron across all trials [fr_neuron0, fr_neuron1, ..., fr_neuronN]
    # shape (N, ) where N is the number of neurons in signal - inlcuding M1 and PMd in total 309 neurons
    fr_neurons = get_average_firing_rates(td_processed, "both_spikes",divide_by_bin_size=None) 
    overall_FR = np.mean(fr_neurons)
    overall_FR_sme = np.std(fr_neurons)/np.sqrt(len(fr_neurons)) 

    num_clusters = len(set(clustering_solution))
    avg_FR = np.zeros((num_clusters,1)) # average firing rate of each cluster
    max_FR = np.zeros((num_clusters,1)) # max firing rate of each cluster
    sme_FR = np.zeros((num_clusters,1)) # sme firing rate of each cluster = std/sqrt(N)
    
    for clust in range(num_clusters): # for each cluster
        fr_cluster = []
        for neuron in range(len(clustering_solution)): # for each neuron
            if(clustering_solution[neuron] == clust): # if neuron is in cluster 'clust'
                fr_cluster.append(fr_neurons[neuron]) # append firing rate of neuron

        fr_cluster = np.array(fr_cluster)
        avg_FR[clust] = np.mean(fr_cluster)
        sme_FR[clust] = np.std(fr_cluster)/math.sqrt(len(fr_cluster))
        max_FR[clust] = np.amax(fr_cluster)
    
    return overall_FR, overall_FR_sme, avg_FR, sme_FR, max_FR


def plot_FR(umap_df, overall_FR, overall_FR_sme, avg_FR, sme_FR, max_FR, plot_umap=True, plot_fr_cluster=True):
    '''
    plot firing rate on UMAP graph and plot firing rate for each cluster
    
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position
    @arg overall_FR - average firing rate among all neurons
    @arg overall_FR_sme - SME of firing rate among all neurons
    @arg avg_FR - average firing rate of each cluster
    @arg sme_FR - SME of average firing rate of each cluster
    @arg max_FR - max firing rate of each cluster
    @arg plot_umap - if True plot firing rate of each neuron on UMAP graph
    @arg plot_fr_cluster - if True plot average firing rate of each cluster separately
    '''
    
    if plot_umap:
        # plot firing rate on UMAP plot
        fr_neurons = get_average_firing_rates(td_processed, "both_spikes",divide_by_bin_size=None) 
        fig, ax = plt.subplots()
        fig.set_size_inches(8, 5)
        feature_label = 'Average FR'
        cmap = sns.color_palette('crest', as_cmap=True)
        scat = plt.scatter(umap_df['x'].tolist(), umap_df['y'].tolist(),alpha = 0.8, c = fr_neurons, cmap = cmap)
        cax = fig.add_axes([0.1, 0.05, 0.8, 0.03])
        cbar = fig.colorbar(scat, cax=cax, orientation='horizontal')
        cbar.set_label(feature_label,labelpad=10,fontsize=16)
        cbar.ax.tick_params(labelsize=16)

        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()
    
    if plot_fr_cluster:
        f, arr = plt.subplots(1,2)
        f.set_size_inches(17,5)
        x_pos = np.arange(len(max_FR))
        arr[0].bar(x_pos,max_FR[:,0], color = ['#5e60ce','#00c49a','#ffca3a','#D81159','#fe7f2d','#7bdff2','#0496ff','#efa6c9'], yerr=sme_FR[:,0] )
        arr[0].spines['right'].set_visible(False)
        arr[0].spines['top'].set_visible(False)
        arr[0].set_ylabel('Max FR (spikes/s)', fontsize=15)
        arr[0].set_xlabel('Cluster')
        
        avg_fr_tot = np.append(avg_FR[:,0], overall_FR)
        sme_fr_tot = np.append(sme_FR[:,0], overall_FR_sme)
        
        x_pos_fr = np.arange(len(avg_FR)+1)
        arr[1].bar(x_pos_fr, avg_fr_tot, color = ['#5e60ce','#00c49a','#ffca3a','#D81159','#fe7f2d','#7bdff2','#0496ff','#efa6c9'], yerr=sme_fr_tot)
        arr[1].spines['right'].set_visible(False)
        arr[1].spines['top'].set_visible(False)
        arr[1].set_ylabel('Average FR (spikes/s)',fontsize=15)
        arr[1].set_xlabel('Cluster (7th is baseline -> avg firing rate of all neurons)')

    return

## Peak to trough

In [10]:
def get_ptt(waveform):
    '''
    measure peak to trough duration on a given waveform
    @arg waveform  - list containing one waveform data
    
    @return d - peak to trough duration
    '''
    min_value = np.min(waveform)
    index_min = np.where(waveform == min_value) # index of min -> argmin

    max_value = np.max(waveform[index_min[0][0]:]) # from index of min and after - check index1 dimensions
    index_max = np.where(waveform[index_min[0][0]:] == np.max(waveform[index_min[0][0]:])) # argmax
    index_max[0][0] = index_max[0][0] + index_min[0][0]

    d = (index_max[0][0] - index_min[0][0])
    return d
            
def peak_to_trough_per_cluster(umap_df, neuron_types):
    '''
    find min point of the waveform and the max after that point
    get their indices so that the time duration between them can be calculated -> peak-to-trough duration
    
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    
    @return distance_mean - average ptt for each cluster 
    @return distance_std - std of ptt for each cluster 
    @return sme - SME of ptt for each cluster 
    '''
    
    ptt_mean = [] # list containing avergae peak-to-trough distance for each cluster
    ptt_std = [] # list containing std peak-to-trough distance for each cluster
    ptt_sme = [] # list containing std/sqrt(n) peak-to-trough distance for each cluster


    # peak to trough for all neurons ordered by clusters
    for cluster in range(len(neuron_types)): # for each cluster
        group_waveforms = umap_df.iloc[neuron_types[cluster]]['waveform'].tolist() # waveform data
        distance = [] # list containing peak-to-trough distance for each neuron

        for i in range(len(group_waveforms)): # for each neuron in this cluster
            waveform = group_waveforms[i]
            d = get_ptt(waveform) * 0.03 # each bin is 0.03sec
            distance.append(d)

        distance = np.array(distance)
        
        d_mean = np.mean(distance)
        ptt_mean.append(d_mean)
        
        d_std = np.std(distance)
        ptt_std.append(d_std)
        
        sme_cluster = np.std(distance)/math.sqrt(len(distance))
        ptt_sme.append(sme_cluster)

    return ptt_mean, ptt_std, ptt_sme


def peak_to_trough_all(data_normalized):
    '''
    compute peak to trough of all neurons (in the order as found in the dataset)
    
    @arg data_normalized - list of all normalized waveforms
    @return list of peak to trough of each neuron
    '''

    ptt_overall = [] # peak to trough across all neurons
    
    for i in range(len(data_normalized)):
        waveform = data_normalized[i]
        d = get_ptt(waveform) * 0.03 # each bin is 0.03sec
        ptt_overall.append(d)
        
    ptt_overall = np.array(ptt_overall)
    
    return ptt_overall

def plot_ptt(umap_df, ptt_overall, ptt_mean, ptt_sme, plot_umap=True, plot_ptt_cluster=True):
    '''
    plot peak-to-trough on UMAP graph and plot peak-to-trough for each cluster
    
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position
    @arg ptt_overall - ptt of each neuron
    @arg ptt_mean - average peak-to-trough of each cluster
    @arg ptt_sme - SME of peak-to-trough of each cluster
    @arg plot_umap - if True plot peak-to-trough of each neuron on UMAP graph
    @arg plot_ptt_cluster - if True plot average peak-to-trough of each cluster separately
    '''
    
    if plot_umap:
        # plot on UMAP graph
        fig, ax = plt.subplots(figsize=(8, 5))
        feature_label = 'Peak to trough distance [ms]'
        cmap = sns.color_palette('crest', as_cmap=True)
        scat = plt.scatter(umap_df['x'].tolist(), umap_df['y'].tolist(),alpha = 0.8,
                            c = ptt_overall, cmap = cmap)
        cax = fig.add_axes([0.1, 0.05, 0.8, 0.03])
        cbar = fig.colorbar(scat, cax=cax, orientation='horizontal')
        cbar.set_label(feature_label,labelpad=10,fontsize=16)
        cbar.ax.tick_params(labelsize=16)

        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()
        
    if plot_ptt_cluster:
        # compare average peak-to-trough between clusters
        ptt_mean = np.array(ptt_mean)
        ptt_sme = np.array(ptt_sme)
        
        ptt_overall_sme = np.std(ptt_overall)/math.sqrt(len(ptt_overall))
        ptt_overall_mean = np.mean(ptt_overall)
        
        mean_ptt = np.append(ptt_mean, ptt_overall_mean)
        sme_ptt = np.append(ptt_sme, ptt_overall_sme)
        
        x_pos_ptt = np.arange(len(mean_ptt))


        f, arr = plt.subplots(figsize=(5, 4))
        arr.bar(x_pos_ptt, mean_ptt, color = ['#5e60ce','#00c49a','#ffca3a','#D81159','#fe7f2d','#7bdff2','#0496ff','#efa6c9'],
                yerr=sme_ptt)
        arr.spines['right'].set_visible(False)
        arr.spines['top'].set_visible(False)
        arr.set_ylabel('Peak to trough duration [ms]', fontsize = 15)
        arr.set_xlabel('Cluster (last is population baseline)')

    return

## AP width

In [3]:
def get_AP_width(waveform):
    '''
    compute AP width of given waveform data
    @arg waveform - waveform data
    
    @return AP width of the given waveform data
    '''
    # the first point is the max point before the global minimum
    # the second one is the global minimum
    # the third one is the max after the global minimum
    # calculate index of all three points
    
    # global min of waveform
    min_value = np.amin(waveform)
    index_min = np.where(waveform == np.amin(waveform[:30])) # index of global min
    index_min = index_min[0][0]

    # max before global min
    max_value = np.amax(waveform[:index_min])
    index_max = np.where(waveform[:index_min] == np.amax(waveform[:index_min]))
    index_max = index_max[0][0]
                                  
    # max after global min
    indexMax = np.where(waveform[index_min:] == np.amax(waveform[index_min:]))
    indexMax = indexMax[0][0] + index_min
    
    height = (max_value + min_value)/2
    found_both = 0
    for index in range(index_max, indexMax):
        if found_both == 2:
            break
        if waveform[index] > height and waveform[index+1] < height:
            index1 = index
            found_both += 1
        if waveform[index] < height and waveform[index+1] > height:
            index2 = index
            found_both += 1

    width = index2 - index1
    
    # plot only to check it's correct
    '''
    plt.plot(waveform, '.')
    plt.grid()
    plt.hlines(height, index1, index2, color='k')
    plt.show()
    '''
    return width
 
    
def AP_width_per_cluster(umap_df, neuron_types):
    '''
    find average AP width of each cluster
    
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    
    @return widths_mean - average AP width for each cluster 
    @return widths_sme - SME of AP width for each cluster 
    '''
    widths_mean = []
    widths_sme = []

    # AP width for all neurons ordered by clusters
    for cluster in range(len(neuron_types)): # for each cluster
        group_waveforms = umap_df.iloc[neuron_types[cluster]]['waveform'].tolist() # all (average) waveforms in this cluster
        widths_cluster = []

        for i in range(len(group_waveforms)): # for each neuron in this cluster
            waveform = group_waveforms[i]
            width = get_AP_width(waveform) * 0.03 # in waveform data each time bin is 0.03 sec
            widths_cluster.append(width)

        widths_mean.append(np.mean(widths_cluster))
        widths_sme.append(np.std(widths_cluster)/math.sqrt(len(group_waveforms)))
    
    return widths_mean, widths_sme


def AP_width_all(data_normalized):
    '''
    compute AP width of all neurons (in the order as found in the dataset)
    
    @arg data_normalized - list of all normalized waveforms
    @return list of AP width of each neuron
    '''
    AP_all = [] # AP width across all neurons

    for i in range(len(data_normalized)):
        waveform = data_normalized[i]
        width = get_AP_width(waveform) * 0.03 # in waveform data each time bin is 0.03 sec
        AP_all.append(width)
        
    AP_all = np.array(AP_all)

    return AP_all


def plot_AP_width(umap_df, AP_all, widths_mean, widths_sme, plot_umap=True, plot_AP_cluster=True):
    '''
    plot Action Potential width on UMAP graph and plot AP width for each cluster
    
    @arg umap_df - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position
    @arg AP_all - AP width of each neuron
    @arg widths_mean - average AP width of each cluster
    @arg widths_sme - SME of AP width of each cluster
    @arg plot_umap - if True plot AP width of each neuron on UMAP graph
    @arg plot_AP_cluster - if True plot average AP width of each cluster separately
    '''
    
    if plot_umap:
        # plot on UMAP graph
        fig, ax = plt.subplots(figsize=(8, 5))
        feature_label = 'Action potential width [ms]'
        cmap = sns.color_palette('crest', as_cmap=True)
        scat = plt.scatter(umap_df['x'].tolist(), umap_df['y'].tolist(),alpha = 0.8,
                            c = AP_all,
                            cmap = cmap )
        cax = fig.add_axes([0.1, 0.05, 0.8, 0.03])
        cbar = fig.colorbar(scat, cax=cax, orientation='horizontal')
        cbar.set_label(feature_label,labelpad=10,fontsize=16)
        cbar.ax.tick_params(labelsize=16)

        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()
        
    if plot_AP_cluster:
        # compare average AP width between clusters
        AP_all_sme = np.std(AP_all)/math.sqrt(len(AP_all))
        AP_all_mean = np.mean(AP_all)
        
        x_pos_ap = np.arange(len(widths_mean)+1)
        AP_widths_mean = np.append(widths_mean, AP_all_mean)
        AP_widths_sme = np.append(widths_sme, AP_all_sme)

        f, arr = plt.subplots(figsize=(5, 4))
        arr.bar(x_pos_ap, AP_widths_mean, color = ['#5e60ce','#00c49a','#ffca3a','#D81159','#fe7f2d','#7bdff2','#0496ff','#efa6c9'],
                yerr=AP_widths_sme)
        arr.spines['right'].set_visible(False)
        arr.spines['top'].set_visible(False)
        arr.set_ylabel('Average AP width [ms]', fontsize = 15)
        arr.set_xlabel('Cluster (last is population baseline)')

    return

## Plot violin

In [ ]:
def plot_violin(neurons_prop, neuron_types, color, ylabel, shift=0, ax=False):
    '''
    plot distribution of physiologival property as violin plots per each cluster
    
    @arg neurons_props - physiological property of each neuron in the dataset
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    @color - list of colors to plot each cluster
    @arg ylabel - ylabel for plot
    @arg shift - not equal 0 if need to compare 2 datasets on the same plot
    @arg ax - axes where to plot graph, if False, generate new axes
    '''
    _neurons_cluster = []

    # select only neurons from specific cluster
    for clust in range(len(neuron_types)):
        _neurons_cluster.append(neurons_prop[neuron_types[clust]]) # mean ptt

    if ax == False:
        fig, ax = plt.subplots()
        
    alpha = 0.6
    if shift < 0: alpha = 0.2
    
    pos = 1
    if shift != 0: pos = 1.5
    
    for clust in range(len(neuron_types)):
        violin = ax.violinplot(_neurons_cluster[clust], positions=[pos*clust+shift], showmedians=True)
       # violin['bodies'].set_facecolor(color[clust])
        violin['bodies'][0].set_facecolor(color[clust])
        violin['cbars'].set_colors(color[clust])
        violin['cmedians'].set_colors(color[clust])
        violin['cmins'].set_colors(color[clust])
        violin['cmaxes'].set_colors(color[clust])
        violin['bodies'][0].set_alpha(alpha)


    clust += 1
    violin = ax.violinplot(neurons_prop, positions=[pos*clust+shift], showmedians=True)
    violin['bodies'][0].set_facecolor('k')
    violin['cbars'].set_colors('k')
    violin['cmedians'].set_colors('k')
    violin['cmins'].set_colors('k')
    violin['cmaxes'].set_colors('k')
    violin['bodies'][0].set_alpha(alpha)
    ax.set_xticks([i*pos for i in [0,1,2,3,4]], labels = ['0','1','2','3','Population']);
    ax.set_ylabel(ylabel)
    
    return 

# Decoders

## Split data 

In [4]:
def split_data(X, y, training_range, testing_range, valid_range, bins_before, bins_current, bins_after, flat, zero_center):
    '''
    split data into training, test and validation
    
    @arg X - input data: spikes with history
    @arg y - output data (what needs to be predicted)
    @arg training_range - part of data used for training
    @arg testing_range - part of data used for testing (used to find optimal parameters)
    @arg valid_range - part of data used for validation
    @arg bins_before - how many bins of neural data prior to the output are used for decoding
    @arg bins_current - whether to use concurrent time bin of neural data
    @arg bins_after - how many bins of neural data after the output are used for decoding
    @arg flat - if True, flatten input data so so each "neuron / time" is a single feature 
                 format for Wiener Filter, Wiener Cascade, XGBoost, Dense Neural Network and GBN
    @arg zero_center - if True, compute zer-centered data

    @return input and output data split into train, test and validation data
    '''
    num_examples = X.shape[0]
    
    #Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
    #This makes it so that the different sets don't include overlapping neural data
    training_set = np.arange(int(np.round(training_range[0]*num_examples))+bins_before, int(np.round(training_range[1]*num_examples))-bins_after)
    testing_set = np.arange(int(np.round(testing_range[0]*num_examples))+bins_before, int(np.round(testing_range[1]*num_examples))-bins_after)
    valid_set = np.arange(int(np.round(valid_range[0]*num_examples))+bins_before, int(np.round(valid_range[1]*num_examples))-bins_after)


    if flat: # flatten
        X = X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

    #Get training data
    y_train = y[training_set,...]
    X_train = X[training_set,...]

    #Get testing data
    y_test = y[testing_set,...]
    X_test = X[testing_set,...]

    #Get validation data
    y_valid = y[valid_set,...]
    X_valid = X[valid_set,...]
    
    #Z-score "X" inputs. 
    X_train_mean = np.nanmean(X_train,axis=0)
    X_train_std = np.nanstd(X_train,axis=0)
    X_train = (X_train-X_train_mean)/X_train_std
    X_test = (X_test-X_train_mean)/X_train_std
    X_valid = (X_valid-X_train_mean)/X_train_std
    
    if zero_center:
        #Zero-center outputs
        y_train_mean = np.mean(y_train,axis=0)
        y_train = y_train-y_train_mean
        y_test = y_test-y_train_mean
        y_valid = y_valid-y_train_mean
        
        
        
    return X_train, X_test, X_valid, y_train, y_test, y_valid


## R2 plot

In [1]:
def plot_R2(r2, r2_all, color='k'):
    '''
    plot average coefficient R2 of each cluster and average R2 when sampled neurons from all dataset
    
    @arg r2 - R2 from each cluster, list NxM, N is number of repetitions, M is number of clusters
    @arg r2_all - list of R2 obtained randomly sampling neurons, size is 1xN
    @arg color - color of the plotted line, default is balck
    
    @return average R2 and SME
    '''
    # R2 from each cluster
    r2_clust_mean = np.mean(r2, axis=0)
    r2_clust_sme = np.std(r2, axis=0)/math.sqrt(np.shape(r2)[0])

    # R2 sampling neurons randomly from dataset
    r2_all_avg = np.mean(r2_all)
    r2_all_sme = np.std(r2_all)/math.sqrt(len(r2_all))

    r2_mean = np.append(r2_clust_mean, r2_all_avg)
    r2_sme = np.append(r2_clust_sme, r2_all_sme)

    clusts = np.arange(np.shape(r2)[1]+1)
    plt.errorbar(clusts, r2_mean, yerr=r2_sme,c=color, marker='o', fillstyle='full', markerfacecolor=color)
    #plt.xlabel('Cluster (last is whole dataset baseline)', fontsize=12)
    plt.ylabel(r'Coefficient of determination, $R^2$', fontsize=12)
    
    return r2_mean, r2_sme

## Velocity decoding vs cluster properties

In [11]:
def plot_R2_vs_property(r2_mean, r2_sme, avg, sme, CUSTOM_PAL_SORT, y_label):
    '''
    plot R2 coefficient from velocity decoding of each cluster vs firing rate
    
    @arg r2_mean - average R2 score for each cluster
    @arg r2_std - std of R2 for each cluster
    @arg avg_fr - average firing rate of each cluster
    @arg sme_fr - SME of firing rate of each cluster
    @arg CUSTOM_PAL_SORT_3 - colour palette 
    @arg y_label - ylabel for plot, 'Coefficient of Determination' or 'Target Classifier Accuracy'
    '''
    f, ax2 = plt.subplots(figsize=(len(r2_mean),5))
    
    clusts = np.arange(len(r2_mean))
    ax1 = ax2.twinx()
    ax1.errorbar(clusts, r2_mean, yerr=r2_sme, marker='o', fillstyle='full', markerfacecolor='w', color='k', ecolor='k')
    ax1.set_ylabel(r'Coefficient of determination, $R^2$')
    ax1.yaxis.label.set_color('k')
    ax1.tick_params(axis='y',colors='k')
    ax1.spines['top'].set_visible(False)
    ax1.spines['left'].set_color('k')
    
    x_pos = np.arange(len(avg))
    ax2.bar(x_pos, avg, color = CUSTOM_PAL_SORT, yerr=sme, ecolor='gray')
    ax2.set_xlabel('Cluster (last is population baseline)')
    ax2.set_ylabel(f'{y_label}',fontsize=12)
    ax2.tick_params(axis='y')
    ax2.spines['top'].set_visible(False)
    
    return

# M1 vs PMd

In [ ]:
# not used
def target_classifier_M1_and_PMd(td_processed, M1_neurons, PMd_neurons, N_samples, pca_dims, rep):
    '''
    implement target classifier on data sampled from M1 and PMd separately
    @arg td_processed - preprocessed and filtered firing data
    @arg M1_neurons - int, number of neurons from M1
    @arg PMd_neurons - int, number of neurons from PMd
    @arg N_samples - number of neurons sampled from M1 or PMd
    @arg pca_dims - number of dimensions for PCA
    @arg rep - number of times to compute analysis with same parameters. At the end compute average
    
    @return accuracies - accuracy obtain for M1 and PMd, list size Nx2, N is number of repetitions
    @return y_valid - true target values (not used)
    @return y_predicitons - predeicted target values (not used)
    '''
    accuracies = np.zeros((rep,2))

    for neurons in ['M1', 'PMd']:
        for z in range(rep):
            print(neurons, z)

            move_td = td_processed.copy()
            y = []
            neural_spikes = []

            if neurons == 'M1':
                pos = 0
                sampled_data = random.sample(list(np.arange(M1_neurons)), N_samples)
                for i in range(move_td.shape[0]):
                    move_td.M1_rates[i] = move_td.M1_rates[i][:, sampled_data]
                move_td = dim_reduce(move_td, PCA(pca_dims), "M1_rates", "M1_pca")
                neural_data = concat_trials(move_td, "M1_pca")

                for i in range(move_td.shape[0]):
                    y.append(move_td.target_id[i])
                    neural_spikes.append(move_td.M1_pca[i])


            elif neurons == 'PMd':
                pos = 1
                sampled_data = random.sample(list(np.arange(PMd_neurons)), N_samples)
                for i in range(move_td.shape[0]):
                    move_td.PMd_rates[i] = move_td.PMd_rates[i][:, sampled_data]
                move_td = dim_reduce(move_td, PCA(pca_dims), "PMd_rates", "PMd_pca")
                neural_data = concat_trials(move_td, "PMd_pca")

                for i in range(move_td.shape[0]):
                    neural_spikes.append(move_td.PMd_pca[i])
                    y.append(move_td.target_id[i])

            # define what time period to use spikes from (with respect to the output)
            bins_before = 6
            bins_current = 1 
            bins_after = 6

            y = np.array(y)
            X = np.array(neural_spikes)

            training_range = [0, 0.7]
            testing_range = [0.7, 0.85]
            valid_range = [0.85,1]

            X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(X, y, training_range, testing_range, valid_range,
                                                                             bins_before, bins_current, bins_after, flat=False, zero_center=False)

            #Declare model
            LSTM_classifier = LSTMClassification(units=100, dropout=0, num_epochs=10)

            #Fit model
            LSTM_classifier.fit(X_train, y_train)

            #Get predictions
            y_predicitons =  LSTM_classifier.predict(X_valid)

            #Get metric of fit
            accuracy = np.sum(y_predicitons == y_valid) / float(len(y_predicitons))
            accuracies[z,pos] = accuracy
            
    return accuracies, y_valid, y_predicitons


In [ ]:
def target_classifier_M1_or_PMd(td_processed, neurons, M1_neuorns, PMd_neurons, N_samples, pca_dims, rep):
     '''
    implement target classifier on data sampled from either M1 or PMd 
    
    @arg td_processed - preprocessed and filtered firing data
    @arg neurons - str: 'M1' or 'PMd', if compute analysis on M1 neurons or on PMd neurons
    @arg M1_neurons - int, number of neurons from M1
    @arg PMd_neurons - int, number of neurons from PMd
    @arg N_samples - number of neurons sampled from M1 or PMd
    @arg pca_dims - number of dimensions for PCA
    @arg rep - number of times to compute analysis with same parameters. At the end compute average
    
    @return accuracies - accuracy obtain for M1 or PMd, list size N, N is number of repetitions
    @return y_valid - true target values (not used)
    @return y_predicitons - predeicted target values (not used)
    '''
    
    accuracies = np.zeros((rep))
    
    print(pca_dims)
    for z in range(rep):
        print(neurons, z)

        move_td = td_processed.copy()
        y = []
        neural_spikes = []

        if neurons == 'M1':
            sampled_data = random.sample(list(np.arange(M1_neurons)), N_samples)
            for i in range(move_td.shape[0]):
                move_td.M1_rates[i] = move_td.M1_rates[i][:, sampled_data]
            move_td = dim_reduce(move_td, PCA(pca_dims), "M1_rates", "M1_pca")
            neural_data = concat_trials(move_td, "M1_pca")

            for i in range(move_td.shape[0]):
                y.append(move_td.target_id[i])
                neural_spikes.append(move_td.M1_pca[i])


        elif neurons == 'PMd':
            sampled_data = random.sample(list(np.arange(PMd_neurons)), N_samples)
            for i in range(move_td.shape[0]):
                move_td.PMd_rates[i] = move_td.PMd_rates[i][:, sampled_data]
            move_td = dim_reduce(move_td, PCA(pca_dims), "PMd_rates", "PMd_pca")
            neural_data = concat_trials(move_td, "PMd_pca")

            for i in range(move_td.shape[0]):
                neural_spikes.append(move_td.PMd_pca[i])
                y.append(move_td.target_id[i])

        # define what time period to use spikes from (with respect to the output)
        bins_before = 6
        bins_current = 1 
        bins_after = 6

        y = np.array(y)
        X = np.array(neural_spikes)

        training_range = [0, 0.7]
        testing_range = [0.7, 0.85]
        valid_range = [0.85,1]

        X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(X, y, training_range, testing_range, valid_range,
                                                                         bins_before, bins_current, bins_after, flat=False, zero_center=False)

        #Declare model
        LSTM_classifier = LSTMClassification(units=100, dropout=0, num_epochs=10)

        #Fit model
        LSTM_classifier.fit(X_train, y_train)

        #Get predictions
        y_predicitons =  LSTM_classifier.predict(X_valid)

        #Get metric of fit
        accuracy = np.sum(y_predicitons == y_valid) / float(len(y_predicitons))
        accuracies[z] = accuracy
            
    return accuracies, y_valid, y_predicitons


In [ ]:
def movement_decoding_M1_or_PMd(td_processed, neurons, M1_neuorns, PMd_neurons, N_samples, pca_dims, repetitions):
     '''
    implement velocity decoder on data sampled from either M1 or PMd 
    
    @arg td_processed - preprocessed and filtered firing data
    @arg neurons - str: 'M1' or 'PMd', if compute analysis on M1 neurons or on PMd neurons
    @arg M1_neurons - int, number of neurons from M1
    @arg PMd_neurons - int, number of neurons from PMd
    @arg N_samples - number of neurons sampled from M1 or PMd
    @arg pca_dims - number of dimensions for PCA
    @arg repetitions - number of times to compute analysis with same parameters. At the end compute average
    
    @return r2 - R2 obtain for M1 or PMd, list size N, N is number of repetitions
    @return y_valid - true target values (not used)
    @return y_predicitons - predeicted target values (not used)
    '''
    r2 = np.zeros((repetitions))
    
    print(pca_dims)
    for rep in range(repetitions):
        print(neurons, rep)

        move_td = td_processed.copy()
        y = []
        neural_spikes = []

        if neurons == 'M1':
            sampled_data = random.sample(list(np.arange(M1_neurons)), N_samples)
            for i in range(move_td.shape[0]):
                move_td.M1_rates[i] = move_td.M1_rates[i][:, sampled_data]
            move_td = dim_reduce(move_td, PCA(pca_dims), "M1_rates", "M1_pca")
            neural_data = concat_trials(move_td, "M1_pca")



        elif neurons == 'PMd':
            sampled_data = random.sample(list(np.arange(PMd_neurons)), N_samples)
            for i in range(move_td.shape[0]):
                move_td.PMd_rates[i] = move_td.PMd_rates[i][:, sampled_data]
            move_td = dim_reduce(move_td, PCA(pca_dims), "PMd_rates", "PMd_pca")
            neural_data = concat_trials(move_td, "PMd_pca")

            
        
        # define what time period to use spikes from (with respect to the output)
        bins_before = 6
        bins_current = 1 
        bins_after = 6


        X = get_spikes_with_history(neural_data, bins_before, bins_after, bins_current)
        y = concat_trials(move_td, "vel")

        training_range = [0, 0.7]
        testing_range = [0.7, 0.85]
        valid_range = [0.85,1]

        X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(X, y, training_range, testing_range, valid_range,
                                                                         bins_before, bins_current, bins_after, flat=False, zero_center=False)

        #Declare model
        model_lstm = LSTMDecoder(units=400,dropout=0,num_epochs=5)

        #Fit model
        model_lstm.fit(X_train,y_train)

        #Get predictions
        y_valid_predicted =  model_lstm.predict(X_valid)

        #Get metric of fit
        R2s_lstm = get_R2(y_valid,y_valid_predicted)
        r2[rep] = np.mean(R2s_lstm)
            
    return r2, y_valid, y_valid_predicted


# Correlation

In [1]:
def sort_by_cluster(fr_correlation_avg, neuron_types, neurons_tot):
    '''
    sort pairwise correlation of all neurons by cluster
    
    @arg fr_correlation_avg - list NxN, where N is the number of neurons
                            each entry is the correlation between neurons i and neuron j
    @arg neuron_types - dict: key is cluster, values are list with all neurons belonging to that cluster
    @arg neurons_tot - int, N, total number of neurons
    
    @return fr_correlation_ordered - list NxN, where N is the number of neurons
                            each entry is the correlation between neurons i and neuron j
                            neurons are ordered by clusters
    '''
    neurons_ordered = [] # contain all neuron indices ordered by cluster they belong to
    for cluster in range(len(neuron_types)):
        neurons_ordered.extend(neuron_types[cluster])

    fr_correlation_ordered = np.zeros((neurons_tot, neurons_tot))
    for idx1 in range(neurons_tot):
        neuron1 = neurons_ordered[idx1]

        for idx2 in range(neurons_tot):
            neuron2 = neurons_ordered[idx2]
            fr_correlation_ordered[idx1, idx2] = fr_correlation_avg[neuron1, neuron2]
    
    return fr_correlation_ordered
    

# Compare datasets

In [ ]:
def match_clusters(umap_df1, umap_df2, neuron_types1, neuron_types2, plot=True):
    '''
    compare waveform data to match clusters of 2 arrays calculating correlation
    compue average waveform og each cluster and compute pairwise correlation between each pair of clusters
    
    @arg umap_df1 - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position (dataset1)
    @arg umap_df1 - pandas DataFrame with normalised waveform data of each point in UMAP
                    graph and corresponding x and y position (dataset2)
    @arg neuron_types1 - dict: key is cluster, values are list with all neurons belonging to that cluster (dataset1)
    @arg neuron_types2 - dict: key is cluster, values are list with all neurons belonging to that cluster (dataset2)
    @arg plot - if True, plot heatmap table of correlation coefficients

    @return correlation - list NxN, where N is the number of clusters
                            each entry is pairwise correlation between cluster i and cluster j
    @return maxValue - list size N, index of best matches
    '''
    # get average waveform of each cluster for each dataset
    df1 = pd.DataFrame() # each column -> average waveform for each cluster
    df2 = pd.DataFrame()
    # each column is a different cluster, each row is a different point in time (values are the average waveform of that cluster)

    for cluster in range(len(neuron_types1)):
        clust_waveforms = umap_df1.iloc[neuron_types1[cluster]]['waveform'].tolist() # all waveforms in this cluster
        df1[f'dataset 1 - cluster {cluster}'] = np.mean(clust_waveforms,axis=0) # mean waveform of each cluster

        clust_waveforms = umap_df2.iloc[neuron_types2[cluster]]['waveform'].tolist() # all waveforms in this cluster
        df2[f'dataset 2 - cluster {cluster}'] = np.mean(clust_waveforms,axis=0) # average waveform of each cluster

    # compute correlation between clusters
    correlation = np.zeros((len(neuron_types1), len(neuron_types2)))
    for i in range(len(neuron_types1)):
        for j in range(len(neuron_types2)):
            correlation[i,j] = df1[df1.columns[i]].corr(df2[df2.columns[j]])

    correlation = pd.DataFrame(correlation)

    # create heatmap for the calculated correlation
    if plot:
        plt.figure(figsize=(5,4))
        sns.heatmap(correlation, annot=True, fmt='.2g')
        #plt.title('Correlation between dataset 15/09/16 and 19/09/16', fontsize=14) # change with new dataset

    # find max value of each row
    maxValue = correlation.idxmax(axis=1) # max value of each row -> matching clusters
    
    return correlation, maxValue

In [1]:
def order_array(arr_unordered, corr_idx, axis=None):
    '''
    reorder array to match order matched clusters
    
    @arg arr_unordered - unordered array, each column is a different cluster
    @arg corr_idx - list containing order of matched clusters
    @arg axis - str: 'col' or 'row',
                if arr_unordered is a 2-dimensional array, need to specify order along which dimension
    
    
    @return ordered array
    '''
    arr_unordered = np.array(arr_unordered)
    arr_ordered = np.zeros((arr_unordered.shape))
    for i, idx in enumerate(corr_idx):
        if np.ndim(arr_unordered) == 2:
            if axis == 'col':
                arr_ordered[:,i] = arr_unordered[:,idx]
            elif axis == 'row':
                arr_ordered[i,:] = arr_unordered[idx,:]
        else:
            arr_ordered[i] = arr_unordered[idx]
    return arr_ordered

def compare_values_plot(ax, arr_mean1, arr_mean2, arr_sme1, arr_sme2, label, plot_sme=False):
    clusts = np.arange(len(arr_mean1))
    
    dashed = (0, (5,5))
    if plot_sme:
        ax.errorbar(clusts, arr_mean1, yerr=arr_sme1, marker='o', fillstyle='full',
                    color='gray', label=f'{label} 1',linestyle=dashed,markersize=5)
        ax.errorbar(clusts, arr_mean2, yerr=arr_sme2, marker='o', fillstyle='full',
                    color='darksalmon', label=f'{label} 2',linestyle=dashed, markersize=5)
    
    else:
        ax.plot(arr_mean1, marker='o', color='gray', label=f'{label} 1',linestyle=dashed,markersize=5)
        ax.plot(arr_mean2, marker='o', color='darksalmon', label=f'{label} 2',linestyle=dashed, markersize=5)
    
    ax.set_xlabel('Cluster')
    ax.set_xticks([0,1,2,3,4], labels = ['0','1','2','3','Population']);
    ax.spines['left'].set_linestyle(dashed)
    ax.spines['top'].set_visible(False)


In [3]:
def div_circ_cmap(data, light_color, dark_color):
    '''
    generate colourmap that is divergent and circular
    
    @arg data - data to build colourmap on
    @arg light_color, dark_color - colors in hex format
    '''
    
    # Define the normalization range
    minn = np.mean(data) - np.std(data)
    maxx = np.mean(data) + np.std(data)
    vmin, vmax = -np.max([abs(minn), maxx]), np.max([abs(minn), maxx])
    #vmin, vmax = np.min(data), np.max(data)
    #vmin, vmax = -1, 1

    # Create a custom normalization that centers zero value
    norm_out = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

    # Define the color map
    cmap_name = 'diverging_circular'
    num_steps = 256

    # Define the color gradient for negative values (dark to light)
    neg_colors = [dark_color, light_color]  # Dark to Light
    neg_cmap = mcolors.LinearSegmentedColormap.from_list(cmap_name + '_neg', neg_colors, num_steps)

    # Define the color gradient for positive values (light to dark)
    pos_colors = [light_color, dark_color]  # Light to Dark
    pos_cmap = mcolors.LinearSegmentedColormap.from_list(cmap_name + '_pos', pos_colors, num_steps)

    # Combine the negative and positive colormaps into a circular colormap
    cmap_out = mcolors.LinearSegmentedColormap.from_list(
        cmap_name, [neg_cmap(0), 'white', pos_cmap(255)], num_steps
    )
    return cmap_out, norm_out


# Save to File

In [ ]:
# save values into files
def save_to_file(r2_vel_dataset1):
    np.savetxt(f'{r2_vel_dataset1}.txt',r2_vel_dataset1)

def load_from_file(file_name1, file_name2):
    r2_vel_dataset1 = np.loadtxt(file_name1)
    r2_all_dataset1 = np.mean(np.loadtxt(file_name2))
    return r2_vel_dataset1, r2_all_dataset1